# Les décorateurs
## Explications

In [ ]:
def tartine(garniture):
    def wrapper(*args, **kwargs):
        price = garniture(*args, **kwargs)
        print("\\________/")

        return price + 1

    return wrapper


def sandwich(garniture):
    def wrapper(*args, **kwargs):
        print("/TTTTTTTT\\")
        price = garniture(*args, **kwargs)
        print("\\________/")

        return price + 1

    return wrapper

@sandwich
def kebab(salade_tomates_oignins:bool):
    if salade_tomates_oignins:
        print("salade_tomates_oignins")
    print('viande')

    return 5

@sandwich
def parisien():
    print("  Jambon ")
    print("  Beurre ")

    return 6

@sandwich
def bagnat():
    print("  thon ")
    print("  salade ")

    return 7


In [ ]:
kebab(True)

In [ ]:
print(parisien())
print(parisien.__name__)

In [ ]:
bagnat()

In [ ]:
sandwich(parisien)()

## Exercices
### Exercice 1

In [ ]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        value = func(*args, **kwargs)
        end = time.time()
        print(end - start, "secondes se sont écoulées")
        return value

    return wrapper

@time_it
def hard_work():
    for x in range(1_000_000):
        y = x ** 2


In [ ]:
hard_work()

### Second exercice

In [ ]:
def count_calls(func):
    counts = 0
    def wrapper(*args, **kwargs):
        nonlocal counts
        counts += 1
        return func(*args, **kwargs)

    def calls():
        return counts

    wrapper.nbcalls = calls

    return wrapper

@count_calls
def some_func():
    print("func called")

@count_calls
def get_data(value):
    return value * 2

print(some_func.nbcalls())
some_func()
some_func()
print(some_func.nbcalls())

### Troisième exercice

In [ ]:
import time
from collections import OrderedDict

def long_calls(func):

    previouses = OrderedDict()

    def inner(value:int):
        try:
            result = previouses[value]
            previouses.move_to_end(value)
        except KeyError:
            result = func(value)
            previouses[value] = result
            if len(previouses) > 5:
                previouses.popitem(last=False)

        return result

    return inner

@time_it
@long_calls
def process(value):
    time.sleep(2)
    return value**2


In [ ]:
print(process(1))
print(process(2))
print(process(3))
print(process(4))
print(process(5))
print(process(6))
print(process(1))
print(process(3))


### Exercice 4 - version de base
Dans les codes suivants, nous utilisons un décorateur *simple*.

In [ ]:
def register(notification):
    targets.append(notification)
    return notification

targets = []

In [ ]:
@register
def notify_mail():
    print("Envoi mail")

def notify_SMS():
    print("Envoi SMS")

@register
def notify_push():
    print("Push information")

In [ ]:

def send_notifications():
    for func in targets:
        func()

send_notifications()

### Exercice 4 - avec niveaux
Cette version utilise des décorateurs paramétrés.

In [ ]:
def register(level):
    def register_decorator(notification):
        targets.append((notification, level))
        return notification
    return register_decorator

targets = []

In [ ]:
@register(level=1)
def notify_mail():
    print("Envoi mail")

@register(level=2)
def notify_SMS():
    print("Envoi SMS")

@register(level=1)
def notify_push():
    print("Push information")

In [ ]:

def send_notifications(level:int=1):
    for func, func_level in targets:
        if func_level <= level:
            func()

send_notifications()

La version ci-dessous du décorateur permet de l'utiliser de manière paramétré ou non. Ainsi, une fonction décorée de la manière suivante :
```python
@register
def notify():
    print("Envoi")
```
Aura le comportement d'une fonction décorée avec `@register(level=1)`.

In [ ]:
def register(_func=None, *, level=1):
    def register_decorator(notification):
        targets.append((notification, level))
        return notification

    if _func is None:
        return register_decorator
    else:
        return register_decorator(_func)
